In [1]:
import matplotlib.pyplot as plt #Plotting
import numpy as np #Number manipulation
import re #String manipulation
import pandas as pd #Dataframe manipulation
import math #Math
import matplotlib #Plotting
from __future__ import division #Math
import seaborn as sns #Better plotting
import scipy #Statistical tests
%matplotlib inline 


In [2]:
subj = 1 #Enter the subject number you want to analyze!

sfile=("a"+str(subj)+".txt")# Assigning file name to variable
with open(sfile, "r") as f: # Opens file in read only('r') mode
    count = 0 #Counter 
    file = "trial" 
    openfile = open(file + str(count), "w") #Assigning to open trial file in write only ('w') mode
    
    for line in f:
        if line.startswith("TRIAL:"):
            openfile.close # Closes the previous trial file
            count +=1 #Adds one to counter
            openfile = open(file + str(count)+".txt", "w") #Opens the next trial file
        else:
            openfile.write(line) # Records line to trial file
    openfile.close()


In [3]:
trials = [] # Creates list for all trials
for i in range(1,101): # Trial 0 includes only general information, no data. It takes all the trials from 1 to 100.
    with open("trial"+str(i)+".txt", "r") as r:
        trials.append(r.read())


In [4]:
edata = [] # List for eye data
mdata = [] # List for mouse data
prefs = [] # List for preferences

for i in trials:
    mouse, eyes = i.split("eyedata",1) # Saves eye and mouse datas seperately
    if "State" in eyes: # Decision making phases/mouse clicks
        eyes, state = eyes.split("State",1)
        button = state.split("button: ")[-1] #Take the last button press as the official preference/can be adjusted
        prefs.append(button[0])
        edata.append(eyes)
        mdata.append(mouse)
    else: # if no mouse click is detected
        prefs.append("9") # No preference symbol (9)
        edata.append(eyes)
        mdata.append(mouse)

In [5]:
for z in range(0,100):
    
    mi=[] #container for a single trial data

    for x in mdata[z].splitlines():
        clean = re.sub('[txy* ]', "", x.replace(":", ",")) # Cleans data to make it csv compatiable
        mi.append(clean[1:])    

    with open('mtrial'+str(z+1)+".csv", 'w') as myfile: #Saves trial data as csv for easier processing
        for item in mi:
             myfile.write("%s\n" % item)

    mheaders = "time","m_x","m_y" # Gives headers to matrix columns for mouse movements
    filename= ("mtrial"+str(z+1)+".csv")
    dfx = pd.read_csv(filename, names=mheaders) # Openning csv file as pandas data frame to add other variables

    elx, ely, erx, ery = [], [], [], [] #containers for raw single trial eye data ( eye left x, eye left y...)
    nelx, nely, nerx, nery = [],[],[],[] #containers for normalized single trial eye data
    aelx, aely, aerx, aery = [], [], [], [] #containers for averaged single trial eye data
    sel, ser = [], [] #containers for pupil size data


    for line in edata[z].splitlines(): #Splitting the data into relevant categories
        try:
            laga, luga = line.split("lx: ")
            
            laga, luga = luga.split(" ly: ")

            elx.append(laga)
            
            laga, luga = luga.split(" lavex:")
            ely.append(laga) 
            
            laga, luga = luga.split(" lavey: ")
            aelx.append(laga)
              
            laga, luga = luga.split(" lnormx: ")
            aely.append(laga)
            
            laga, luga = luga.split(" lnormy: ")
            nelx.append(laga)
            
            laga, luga = luga.split(" pupilsizeL: ")
            nely.append(laga)
            
            laga, luga = luga.split(" rx: ")
            sel.append(laga)
            
            laga,luga = luga.split(" ry: ")
            erx.append(laga)
            
            laga,luga = luga.split(" ravex: ")
            ery.append(laga)
            
            laga,luga = luga.split(" ravey: ")
            aerx.append(laga)
            
            laga, luga = luga.split(" rnormx: ")
            aery.append(laga)
            
            laga, luga = luga.split(" rnormy: ")
            nerx.append(laga)
            
            laga, luga = luga.split(" pupilsizeR: ")
            nery.append(laga)
            ser.append(luga)
            
        except:
            #print("Processing ***")
            pass
        
    
    estate = [] #Mapping timesteps to experiment states
    
    for q in range(len(elx)):
        if q < 200:
            estate.append("Cross")
        elif q >= 200 and q<600:
            estate.append("Context")
        elif q >= 600 and q<800:
            estate.append("Cross")
        elif q >= 800 and q<1000:
            estate.append("Images")
        elif q >= 1000 and q<1300:
            estate.append("Empty")
        elif q >= 1300 and q<1315:
            estate.append("Go Signal")
        elif q >= 1315 and q<2000:
            estate.append("Decision")

    #Changing data types to float

    elx = [float(i) for i in elx]
    ely = [float(i) for i in ely]
    erx = [float(i) for i in erx]
    ery = [float(i) for i in ery]
    nelx = [float(i) for i in nelx]
    nely = [float(i) for i in nely]
    nerx = [float(i) for i in nerx]
    nery = [float(i) for i in nery]
    aelx = [float(i) for i in aelx]
    aely = [float(i) for i in aely]
    aerx = [float(i) for i in aerx]
    aery = [float(i) for i in aery]
    sel = [float(i) for i in sel]
    ser = [float(i) for i in ser]
    
    dfx['state'] = estate #Adding data to dataframe structure for easier processing
    dfx['elx'] = elx
    dfx['ely'] = ely
    dfx['erx'] = erx
    dfx['ery'] = ery
    dfx['nelx'] = nelx
    dfx['nely'] = nely
    dfx['nerx'] = nerx
    dfx['nery'] = nery
    dfx['aelx'] = aelx
    dfx['aely'] = aely
    dfx['aerx'] = aerx
    dfx['aery'] = aery
    dfx['sel'] = sel
    dfx['ser'] = ser
    

    dfx = dfx[dfx.elx != 0] #Eliminating blinks/artifacts
    dfx = dfx[dfx.ely != 0]
    dfx = dfx[dfx.erx != 0]
    dfx = dfx[dfx.ery != 0]
    dfx = dfx[dfx.sel < 25 ] 
    dfx = dfx[dfx.ser < 25 ]
    dfx = dfx[dfx.sel >13]
    dfx = dfx[dfx.ser >13]
    
    dfx["xmean"] = (dfx["erx"]+ dfx["elx"])/2
    dfx["ymean"] = (dfx["ery"]+ dfx["ely"])/2
    
    pd.DataFrame.to_csv(dfx, path_or_buf=('sub ' + str(subj)+ ' trial ' + str(z+1)+ '.csv'))
    
    print('Trial ' + str(z+1)+ ' processed!' + " Shape: " + str(dfx.shape))

Trial 1 processed! Shape: (213, 20)
Trial 2 processed! Shape: (1257, 20)
Trial 3 processed! Shape: (1283, 20)
Trial 4 processed! Shape: (1214, 20)
Trial 5 processed! Shape: (1283, 20)
Trial 6 processed! Shape: (1214, 20)
Trial 7 processed! Shape: (1281, 20)
Trial 8 processed! Shape: (1188, 20)
Trial 9 processed! Shape: (1095, 20)
Trial 10 processed! Shape: (1150, 20)
Trial 11 processed! Shape: (1190, 20)
Trial 12 processed! Shape: (921, 20)
Trial 13 processed! Shape: (1243, 20)
Trial 14 processed! Shape: (1089, 20)
Trial 15 processed! Shape: (1171, 20)
Trial 16 processed! Shape: (1040, 20)
Trial 17 processed! Shape: (1140, 20)
Trial 18 processed! Shape: (1152, 20)
Trial 19 processed! Shape: (1050, 20)
Trial 20 processed! Shape: (1280, 20)
Trial 21 processed! Shape: (1046, 20)
Trial 22 processed! Shape: (1164, 20)
Trial 23 processed! Shape: (1226, 20)
Trial 24 processed! Shape: (565, 20)
Trial 25 processed! Shape: (1063, 20)
Trial 26 processed! Shape: (641, 20)
Trial 27 processed! Shape

In [6]:
preft = [] #Containers for preferences
prefx = []

for i in prefs: #Mapping mouse buttons to text for automized graph creation
    if i=="9":
        preft.append("None")
    elif i=="0":
        preft.append("Left")
    else:
        preft.append("Right")

for i in prefs: #Mapping mouse buttons for better graph creation
    if i == "9":
        prefx.append("None")
    elif i == "0":
        prefx.append(-1)
    else:
        prefx.append(1)

        
for i in range(1,100):
    print (i, preft[i])

1 Right
2 Right
3 Left
4 Right
5 Left
6 Right
7 None
8 Right
9 Left
10 Left
11 None
12 Right
13 Right
14 Left
15 Right
16 Right
17 Right
18 Left
19 Left
20 Left
21 Left
22 Right
23 Right
24 Right
25 Left
26 Left
27 Left
28 Left
29 Left
30 Right
31 Left
32 Left
33 Left
34 Left
35 Right
36 Right
37 Left
38 Right
39 Left
40 Right
41 Left
42 Right
43 Left
44 Left
45 Left
46 Left
47 Left
48 Right
49 Left
50 Right
51 Left
52 Left
53 Left
54 Right
55 Right
56 Left
57 Right
58 Left
59 Right
60 Left
61 Left
62 Left
63 Right
64 Left
65 Left
66 Right
67 Left
68 Right
69 Right
70 Right
71 Right
72 Left
73 Right
74 Left
75 Left
76 Left
77 Right
78 None
79 Left
80 Right
81 Left
82 Right
83 Left
84 Left
85 Right
86 Left
87 Right
88 Left
89 Left
90 Left
91 Left
92 Right
93 Left
94 Right
95 Left
96 Right
97 Right
98 Left
99 None


In [7]:
Im_Right_Context = []
Im_Left_Context = []

for i in range(1,101):    
    t = i
    tr = pd.read_csv("sub " + str(subj) + " trial " + str(t) + ".csv")
    
    dfleft = tr[tr.xmean < 640]; dfleft
    dfright = tr [tr.xmean >640] ; dfright

    dfContextleft = dfleft.loc[dfleft['state'] == 'Context']
    #dfimagesleft = dfimagesleft[dfimagesleft.time>8500]
    dfContextright = dfright.loc[dfright['state'] == 'Context']
    #dfimagesright = dfimagesright[dfimagesright.time>8500]

    Total_Left_Context = len(dfContextleft.xmean)
    Total_Right_Context = len(dfContextright.xmean)
    Total_Context = len(dfContextleft.xmean) + len(dfContextright.xmean)

    #Total_Left_Images = dfimagesleft["xmean"].tolist()
    #Total_Right_Images = dfimagesright["xmean"].tolist()



    if Total_Context < 5:
        Im_Left_Context.append("None")  #Marking trials with less than optimal data points
        Im_Right_Context.append("None")

    else:
        PercentageLeftContext = Total_Left_Context / Total_Context
        PercentageRightContext =  Total_Right_Context / Total_Context
        Im_Left_Context.append(PercentageLeftContext)
        Im_Right_Context.append(PercentageRightContext)
    #df_left = ndfx[ndfx['choice'] == -1] 
    #df_right =ndfx[ndfx['choice'] == 1]

ndfc = pd.DataFrame() #Adding probabilities to dataframe for easier plotting
ndfc["choice"] = prefx
ndfc["Im_Right_Context"] = Im_Right_Context
ndfc["Im_Left_Context"] = Im_Left_Context

ndfxc = ndfc[(ndfc["choice"]!="None")] #Cleaning out trials with less than optimal data points
#ndfxe = ndfxe[(ndfxe["Im_Right"]!= "None")]
#ndfxe = ndfxe[(ndfxe["Im_Left"] != "None")]

df_left_con = ndfxc[ndfxc['choice'] == -1] #Seperating probabilities according to choice
df_right_con =ndfxc[ndfxc['choice'] == 1]

In [8]:
Im_Right = []
Im_Left = []

for i in range(1,101):    
    t = i
    tr = pd.read_csv("sub " + str(subj) + " trial " + str(t) + ".csv")
    
    dfleft = tr[tr.xmean < 640]; dfleft
    dfright = tr [tr.xmean >640] ; dfright

    dfimagesleft = dfleft.loc[dfleft['state'] == 'Images']
    dfimagesleft = dfimagesleft[dfimagesleft.time>8500]
    dfimagesright = dfright.loc[dfright['state'] == 'Images']
    dfimagesright = dfimagesright[dfimagesright.time>8500]

    Total_Left_Images = len(dfimagesleft.xmean)
    Total_Right_Images = len(dfimagesright.xmean)
    Total_Images = len(dfimagesleft.xmean) + len(dfimagesright.xmean)

    #Total_Left_Images = dfimagesleft["xmean"].tolist()
    #Total_Right_Images = dfimagesright["xmean"].tolist()



    if Total_Images < 5:
        Im_Left.append("None")  #Marking trials with less than optimal data points
        Im_Right.append("None")

    else:
        PercentageLeftImages = Total_Left_Images / Total_Images
        PercentageRightImages =  Total_Right_Images / Total_Images
        Im_Left.append(PercentageLeftImages)
        Im_Right.append(PercentageRightImages)
    #df_left = ndfx[ndfx['choice'] == -1] 
    #df_right =ndfx[ndfx['choice'] == 1]

ndfi = pd.DataFrame() #Adding probabilities to dataframe for easier plotting
ndfi["choice"] = prefx
ndfi["Im_Right"] = Im_Right
ndfi["Im_Left"] = Im_Left

ndfxi = ndfi[(ndfi["choice"]!="None")] #Cleaning out trials with less than optimal data points
#ndfxi = ndfxi[(ndfxi["Im_Right"]!= "None")]
#ndfxi = ndfxi[(ndfxi["Im_Left"] != "None")]

df_left_im = ndfxi[ndfxi['choice'] == -1] #Seperating probabilities according to choice
df_right_im =ndfxi[ndfxi['choice'] == 1]



In [9]:
Im_Right_Empty = []
Im_Left_Empty = []

for i in range(1,101):    
    t = i
    tr = pd.read_csv("sub " + str(subj) + " trial " + str(t) + ".csv")
    
    dfleft = tr[tr.xmean < 640]; dfleft
    dfright = tr [tr.xmean >640] ; dfright

    dfemptyleft = dfleft.loc[dfleft['state'] == 'Empty']
    #dfimagesleft = dfimagesleft[dfimagesleft.time>8500]
    dfemptyright = dfright.loc[dfright['state'] == 'Empty']
    #dfimagesright = dfimagesright[dfimagesright.time>8500]

    Total_Left_Empty = len(dfemptyleft.xmean)
    Total_Right_Empty = len(dfemptyright.xmean)
    Total_Empty = len(dfemptyleft.xmean) + len(dfemptyright.xmean)

    #Total_Left_Images = dfimagesleft["xmean"].tolist()
    #Total_Right_Images = dfimagesright["xmean"].tolist()



    if Total_Empty < 5:
        Im_Left_Empty.append("None")  #Marking trials with less than optimal data points
        Im_Right_Empty.append("None")

    else:
        PercentageLeftEmpty = Total_Left_Empty / Total_Empty
        PercentageRightEmpty =  Total_Right_Empty / Total_Empty
        Im_Left_Empty.append(PercentageLeftEmpty)
        Im_Right_Empty.append(PercentageRightEmpty)
    #df_left = ndfx[ndfx['choice'] == -1] 
    #df_right =ndfx[ndfx['choice'] == 1]

ndfe = pd.DataFrame() #Adding probabilities to dataframe for easier plotting
ndfe["choice"] = prefx
ndfe["Im_Right_Empty"] = Im_Right_Empty
ndfe["Im_Left_Empty"] = Im_Left_Empty

ndfxe = ndfe[(ndfe["choice"]!="None")] #Cleaning out trials with less than optimal data points
#ndfxe = ndfxe[(ndfxe["Im_Right"]!= "None")]
#ndfxe = ndfxe[(ndfxe["Im_Left"] != "None")]

df_left_em = ndfxe[ndfxe['choice'] == -1] #Seperating probabilities according to choice
df_right_em =ndfxe[ndfxe['choice'] == 1]

In [12]:
print (PercentageLeftImages)
print (PercentageRightImages)

print (PercentageLeftEmpty)
print (PercentageRightEmpty)

0.5490196078431373
0.5490196078431373
0.9848484848484849
0.015151515151515152


In [9]:
scipy.stats.pearsonr(df_left_im.Im_Right, df_right_im.Im_Right)
#Percentage of Looking at Right

(-0.1537102173251183, 0.2916643959584576)

In [18]:
scipy.stats.pearsonr(df_left_im.Im_Left, df_right_im.Im_Left)

#Percentage of Looking at Left
#(Pearson’s correlation coefficient, 2-tailed p-value)


(-0.15371021732511822, 0.2916643959584576)